In [3]:
!pip -q install transformers accelerate datasets


In [4]:
import json
import pandas as pd

FILE_10K = "/content/reviews_10000.jsonl"

rows = []
with open(FILE_10K, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        rows.append({
            "review_id": obj.get("review_id"),
            "stars": obj.get("stars"),
            "text": obj.get("text")
        })

df = pd.DataFrame(rows).dropna(subset=["text", "stars"]).reset_index(drop=True)
print("Shape:", df.shape)
df.head(3)


Shape: (10000, 3)


,review_id,stars,text
0,J5Q1gH4ACCj6CtQG7Yom7g,2,Went for lunch and found that my burger was me...
1,HlXP79ecTquSVXmjM10QxQ,5,I needed a new tires for my wife's car. They h...
2,JBBULrjyGx6vHto2osk_CQ,5,Jim Woltman who works at Goleta Honda is 5 sta...


In [5]:
def stars_to_polarity(stars):
    if stars <= 2:
        return "negative"
    if stars >= 4:
        return "positive"
    return None

df["gold"] = df["stars"].apply(stars_to_polarity)
df_bin = df.dropna(subset=["gold"]).reset_index(drop=True)

print("Nb total:", len(df), "| Nb binaire:", len(df_bin))
df_bin[["stars", "gold", "text"]].head(3)


Nb total: 10000 | Nb binaire: 8998


,stars,gold,text
0,2,negative,Went for lunch and found that my burger was me...
1,5,positive,I needed a new tires for my wife's car. They h...
2,5,positive,Jim Woltman who works at Goleta Honda is 5 sta...


In [6]:
from transformers import pipeline

MODEL_NAME = "google/flan-t5-small"

llm = pipeline("text-generation", model=MODEL_NAME)

print("Modèle chargé :", MODEL_NAME)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/190 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'CwmForCausalLM', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausa

Modèle chargé : google/flan-t5-small


In [7]:
def predict_polarity_zero_shot(review_text):
    prompt = (
        "Classify the sentiment of this review as positive or negative.\n"
        "Answer with only one word: positive or negative.\n\n"
        f"Review: {review_text}\n"
        "Answer:"
    )
    out = llm(prompt, max_new_tokens=5)[0]["generated_text"].strip().lower()
    # Nettoyage simple
    if "positive" in out:
        return "positive"
    if "negative" in out:
        return "negative"
    return "unknown"

# Test sur une phrase
test_text = "The food was great but the service was slow."
print("Texte:", test_text)
print("Prédiction:", predict_polarity_zero_shot(test_text))


Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=5) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Texte: The food was great but the service was slow.
Prédiction: positive


In [8]:
def predict_rating_zero_shot(review_text):
    prompt = (
        "Predict the star rating (1, 2, 3, 4, or 5) for this customer review.\n"
        "Answer with only one number: 1, 2, 3, 4, or 5.\n\n"
        f"Review: {review_text}\n"
        "Answer:"
    )
    out = llm(prompt, max_new_tokens=5)[0]["generated_text"].strip()

    # on extrait le premier chiffre 1-5 trouvé
    for ch in out:
        if ch in "12345":
            return int(ch)
    return None

# Test rapide sur une phrase
test_text = "The food was great but the service was slow."
print("Texte:", test_text)
print("Note prédite:", predict_rating_zero_shot(test_text))


Both `max_new_tokens` (=5) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Texte: The food was great but the service was slow.
Note prédite: 1


plus puissant

In [9]:
from transformers import pipeline

MODEL_NAME = "google/flan-t5-base"

llm_base = pipeline(
    "text-generation",
    model=MODEL_NAME
)

print("Modèle chargé :", MODEL_NAME)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'CwmForCausalLM', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausa

Modèle chargé : google/flan-t5-base


In [10]:
#google/flan-t5-base
def predict_polarity_base(review_text):
    prompt = (
        "You are a strict sentiment classifier.\n"
        "Your task is to decide if the overall sentiment of the review is positive or negative.\n\n"
        "Rules:\n"
        "- If the review contains mostly complaints, bad experiences, or dissatisfaction → negative.\n"
        "- If the review contains mostly praise, satisfaction, or good experience → positive.\n"
        "- If the review is mixed, choose the dominant sentiment.\n\n"
        "You must answer with ONLY one word:\n"
        "positive OR negative\n\n"
        f"Review:\n{review_text}\n\n"
        "Final answer:"
    )

    output = llm_base(prompt, max_new_tokens=5)[0]["generated_text"].strip().lower()

    if "positive" in output:
        return "positive"
    if "negative" in output:
        return "negative"
    return "unknown"


# Test
test_text = "I don't like this restaurant"
print("Texte:", test_text)
print("Prédiction:", predict_polarity_base(test_text))


Texte: I don't like this restaurant


Both `max_new_tokens` (=5) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Prédiction: positive


In [11]:
#google/flan-t5-base
def predict_rating_base(review_text):
    prompt = (
        "You are a strict rating predictor.\n"
        "Your task is to assign a star rating from 1 to 5 based on the overall sentiment.\n\n"
        "Rating scale:\n"
        "1 = extremely negative experience\n"
        "2 = mostly negative\n"
        "3 = mixed or neutral\n"
        "4 = mostly positive\n"
        "5 = extremely positive\n\n"
        "Consider the overall tone, not just one sentence.\n"
        "If complaints dominate → low rating.\n"
        "If praise dominates → high rating.\n\n"
        "You must answer with ONLY one number: 1, 2, 3, 4, or 5.\n\n"
        f"Review:\n{review_text}\n\n"
        "Final rating:"
    )

    output = llm_base(prompt, max_new_tokens=5)[0]["generated_text"].strip()

    for ch in output:
        if ch in "12345":
            return int(ch)
    return None

# Test
test_text = "I don't like this restaurant"
print("Texte:", test_text)
print("Prédiction:", predict_polarity_base(test_text))
print("Note prédite:", predict_rating_base(test_text))


Texte: I don't like this restaurant


Both `max_new_tokens` (=5) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Prédiction: positive


Both `max_new_tokens` (=5) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Note prédite: 1


In [12]:
def predict_rating_base(review_text):
    prompt = (
        "You are a strict rating predictor.\n"
        "Your task is to assign a star rating from 1 to 5 based on the overall sentiment.\n\n"

        "Rating scale:\n"
        "1 = extremely negative experience\n"
        "2 = mostly negative\n"
        "3 = mixed or neutral\n"
        "4 = mostly positive\n"
        "5 = extremely positive\n\n"

        "Polarity ↔ Rating consistency rule:\n"
        "- If sentiment is NEGATIVE → rating must be 1 or 2.\n"
        "- If sentiment is NEUTRAL or mixed → rating must be 3.\n"
        "- If sentiment is POSITIVE → rating must be 4 or 5.\n\n"

        "Examples:\n\n"

        "Review: The food was terrible and the staff were rude.\n"
        "Final rating: 1\n\n"

        "Review: The service was slow and the meal was disappointing.\n"
        "Final rating: 2\n\n"

        "Review: It was okay, nothing special.\n"
        "Final rating: 3\n\n"

        "Review: The food was good but service could improve.\n"
        "Final rating: 4\n\n"

        "Review: Absolutely amazing experience, perfect in every way!\n"
        "Final rating: 5\n\n"

        "Important:\n"
        "- Consider the OVERALL tone.\n"
        "- If complaints dominate → low rating.\n"
        "- If praise dominates → high rating.\n"
        "- Mixed opinions → 3.\n\n"

        "You must answer with ONLY one number: 1, 2, 3, 4, or 5.\n\n"

        f"Review:\n{review_text}\n\n"
        "Final rating:"
    )

    output = llm_base(prompt, max_new_tokens=5)[0]["generated_text"].strip()

    for ch in output:
        if ch in "12345":
            return int(ch)

    return None
    # Test
test_text = "I like this restaurant"
print("Texte:", test_text)
print("Prédiction:", predict_polarity_base(test_text))
print("Note prédite:", predict_rating_base(test_text))


Texte: I like this restaurant


Both `max_new_tokens` (=5) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Prédiction: positive


Both `max_new_tokens` (=5) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Note prédite: 1


In [13]:
import random
from tqdm import tqdm
import pandas as pd

# Pour afficher les phrases complètes
pd.set_option("display.max_colwidth", None)

# On prend 100 avis aléatoires
sample_100 = df.sample(100, random_state=42).reset_index(drop=True)

results = []

for _, row in tqdm(sample_100.iterrows(), total=100):
    text = row["text"]

    polarity = predict_polarity_base(text)
    rating = predict_rating_base(text)

    results.append({
        "review_id": row["review_id"],
        "true_stars": row["stars"],
        "pred_polarity": polarity,
        "pred_rating": rating,
        "text": text
    })

results_df = pd.DataFrame(results)

# Afficher les 10 premières lignes avec phrases complètes
results_df[["true_stars", "pred_rating", "pred_polarity", "text"]].head(10)


  0%|          | 0/100 [00:00<?, ?it/s]Both `max_new_tokens` (=5) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=5) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
  1%|          | 1/100 [00:02<03:52,  2.35s/it]Both `max_new_tokens` (=5) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=5) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https

,true_stars,pred_rating,pred_polarity,text
0,5,1,positive,Skip the starbucks. This is real stuff like that is high end. Go explore your taste buds.
1,2,1,positive,I placed an order that was confirmed for delivery only for it to never arrive. I called them 3-4 times before someone finally answered and it took them a week to give me a refund for the flowers my friend never received. Really disappointing.
2,5,1,positive,"They have cleaned our house 3 consecutive times over 6 weeks. The most recent visit was the best. I was late to work bc the babysitter didn't show up (our kids ages are 2, 4, 6). I left breakfast dishes on the counter, pajamas lying on the floor, etc as I ran out of the house. \n\nWhen I returned that evening, they had taken care of all the messes I had made in the rush. For a morning to be so crazy, it was an awesome evening with a perfectly clean house."
3,4,1,positive,Tiny place but delicious food. If you're willing to wait for something really tasty and comfortable in an informal environment I'd definitely suggest going here. We met up for an early lunch and spent awhile chatting and catching up while we ate and it was great.
4,5,1,positive,This place is amazing!!!!! Decor is super cool! Bartenders are great. The baguettes are to die for! As soon as we arrived we were given a quick tour (which was appreciated). The vibe overall was just So fun! We can't go back for another girls night.
5,5,1,positive,"Nicole and Ethan are awesome leasing/management personnel. Perfect location. If you can't handle elevators, noise, APARTMENT LIVING..... move into a house, leave your bias reviews in your pocket. Otherwise this building has a ton of social and friendly people. Lots of dogs in this area with a park right outside. Pool deck on the roof with a social crowd. Great place for young professionals."
6,4,1,positive,"It was dinner date night with my darling and we picked LT. It's been around for quite a while now but it was our first visit. Of course, I reviewed the menu online and read a few reviews to get a feel of things. \n\nWe ordered two apps that were specials for the evening. I wanted to try the marrow bone because I've only had it roasted, never had ""smoked"" marrow. Since we were both getting a meat entree we decided to go lite for apps. We had the grilled shrimp on a salad of watermelon, cucumber, peach, blueberries and arugla; and an heirloom tomato salad. Both were very good and just right for an app. For entrees my wife had the rack of lamb and I had the pork loin. Again both were very good (so happy to find a restaurant that can cook pork correctly!). If there was a quibble it would be the lamb little was more rare than medium rare but not enough to send back. More a personal preference. \n\nDesert was the bananas foster bread pudding...fantastic. \n\nAll in all, good service and a very good dinner. We'll visit again for sure."
7,5,1,positive,Great for semi private event. Carrie was extremely responsive and professional and made sure we were taken care of. Highly recommend
8,5,1,positive,"It was time to have a genuine Honduran restaurant with quality modern cuisine in New Orleans. The food and service were great, price is reasonable for the concept and location. I ordered the baleada, horchata and a latte, delicious! We will definitely be back! Awesome restaurant!"
9,2,1,positive,Had a Tucson scramble. It was good but the potatoes that came on the side were very old and very cold. They never bothered to check on me and offered no replacement or discount!


In [14]:
# Afficher les 10 premières lignes avec phrases complètes
results_df[["true_stars", "pred_rating", "pred_polarity", "text"]].head(50)

,true_stars,pred_rating,pred_polarity,text
0,5,1,positive,Skip the starbucks. This is real stuff like that is high end. Go explore your taste buds.
1,2,1,positive,I placed an order that was confirmed for delivery only for it to never arrive. I called them 3-4 times before someone finally answered and it took them a week to give me a refund for the flowers my friend never received. Really disappointing.
2,5,1,positive,"They have cleaned our house 3 consecutive times over 6 weeks. The most recent visit was the best. I was late to work bc the babysitter didn't show up (our kids ages are 2, 4, 6). I left breakfast dishes on the counter, pajamas lying on the floor, etc as I ran out of the house. \n\nWhen I returned that evening, they had taken care of all the messes I had made in the rush. For a morning to be so crazy, it was an awesome evening with a perfectly clean house."
3,4,1,positive,Tiny place but delicious food. If you're willing to wait for something really tasty and comfortable in an informal environment I'd definitely suggest going here. We met up for an early lunch and spent awhile chatting and catching up while we ate and it was great.
4,5,1,positive,This place is amazing!!!!! Decor is super cool! Bartenders are great. The baguettes are to die for! As soon as we arrived we were given a quick tour (which was appreciated). The vibe overall was just So fun! We can't go back for another girls night.
5,5,1,positive,"Nicole and Ethan are awesome leasing/management personnel. Perfect location. If you can't handle elevators, noise, APARTMENT LIVING..... move into a house, leave your bias reviews in your pocket. Otherwise this building has a ton of social and friendly people. Lots of dogs in this area with a park right outside. Pool deck on the roof with a social crowd. Great place for young professionals."
6,4,1,positive,"It was dinner date night with my darling and we picked LT. It's been around for quite a while now but it was our first visit. Of course, I reviewed the menu online and read a few reviews to get a feel of things. \n\nWe ordered two apps that were specials for the evening. I wanted to try the marrow bone because I've only had it roasted, never had ""smoked"" marrow. Since we were both getting a meat entree we decided to go lite for apps. We had the grilled shrimp on a salad of watermelon, cucumber, peach, blueberries and arugla; and an heirloom tomato salad. Both were very good and just right for an app. For entrees my wife had the rack of lamb and I had the pork loin. Again both were very good (so happy to find a restaurant that can cook pork correctly!). If there was a quibble it would be the lamb little was more rare than medium rare but not enough to send back. More a personal preference. \n\nDesert was the bananas foster bread pudding...fantastic. \n\nAll in all, good service and a very good dinner. We'll visit again for sure."
7,5,1,positive,Great for semi private event. Carrie was extremely responsive and professional and made sure we were taken care of. Highly recommend
8,5,1,positive,"It was time to have a genuine Honduran restaurant with quality modern cuisine in New Orleans. The food and service were great, price is reasonable for the concept and location. I ordered the baleada, horchata and a latte, delicious! We will definitely be back! Awesome restaurant!"
9,2,1,positive,Had a Tucson scramble. It was good but the potatoes that came on the side were very old and very cold. They never bothered to check on me and offered no replacement or discount!


In [15]:
from sklearn.metrics import accuracy_score

# Polarité (comparaison binaire)
def stars_to_binary(stars):
    if stars <= 2:
        return "negative"
    if stars >= 4:
        return "positive"
    return None

results_df["true_polarity"] = results_df["true_stars"].apply(stars_to_binary)

# On enlève les neutres pour évaluer la polarité
eval_pol = results_df.dropna(subset=["true_polarity"])

pol_acc = accuracy_score(eval_pol["true_polarity"], eval_pol["pred_polarity"])

# Accuracy note exacte
eval_rating = results_df.dropna(subset=["pred_rating"])
rating_acc = accuracy_score(
    eval_rating["true_stars"].astype(int),
    eval_rating["pred_rating"].astype(int)
)

print("Accuracy Polarité (100 avis):", pol_acc)
print("Accuracy Rating exact (100 avis):", rating_acc)


Accuracy Polarité (100 avis): 0.6989247311827957
Accuracy Rating exact (100 avis): 0.2


In [16]:
from sklearn.metrics import classification_report, accuracy_score

# 1. Traitement des données pour la Polarité
def stars_to_binary(stars):
    if stars <= 2:
        return "negative"
    if stars >= 4:
        return "positive"
    return None

results_df["true_polarity"] = results_df["true_stars"].apply(stars_to_binary)





In [17]:
# On filtre les neutres
eval_pol = results_df.dropna(subset=["true_polarity"])

# 2. Affichage des résultats pour la Polarité
print(f"Accuracy polarité : {accuracy_score(eval_pol['true_polarity'], eval_pol['pred_polarity']):.3f}")
print("Classification report (polarité) :\n")
print(classification_report(eval_pol["true_polarity"], eval_pol["pred_polarity"]))

print("-" * 60)

Accuracy polarité : 0.699
Classification report (polarité) :

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        28
    positive       0.70      1.00      0.82        65

    accuracy                           0.70        93
   macro avg       0.35      0.50      0.41        93
weighted avg       0.49      0.70      0.58        93

------------------------------------------------------------


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
# 3. Affichage des résultats pour les Notes (1 à 5 étoiles)
# On s'assure d'avoir des entiers pour la comparaison
y_true_stars = results_df["true_stars"].astype(int)
y_pred_stars = results_df["pred_rating"].astype(int)

print(f"Accuracy globale des notes : {accuracy_score(y_true_stars, y_pred_stars):.2f}")
print("Classification report détaillé par nombre d'étoiles :\n")
print(classification_report(y_true_stars, y_pred_stars))

Accuracy globale des notes : 0.20
Classification report détaillé par nombre d'étoiles :

              precision    recall  f1-score   support

           1       0.20      1.00      0.33        20
           2       0.00      0.00      0.00         8
           3       0.00      0.00      0.00         7
           4       0.00      0.00      0.00        22
           5       0.00      0.00      0.00        43

    accuracy                           0.20       100
   macro avg       0.04      0.20      0.07       100
weighted avg       0.04      0.20      0.07       100



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
